# GSON : Programmation Haute Performance

## Partie 1 : les communications point-à-point

### 1.1 Le paradigme de programmation

MPI est une bibliothèque de fonctions de communications qui permet d'écrire des programmes paralléles. C'est également un environnement d'exécution qui fournit la manière de lancer ce programme parallèle sur une machine à mémoire distribuée.

Avec MPI on fait du parallélisme explicite. Autrement dit celui qui écrit le code doit explicitement gérer la parallélisation et exprimer ce que doit faire chaque processus. De plus il s'agit d'un modèle **SPMD, Single Programme Multiple Data** où on écrit un seul programme qui sera exécuté par tous les processus. 

Dans la suite on va progressivement construire un programme MPI en introduisant les fonctions de communication point-à-point qui permettent d'échanger (envoi/réception) des données entre 2 processus.



Nous allons utiliser **mpi4py** une bibliothèque python qui est une interface vers le standard MPI. Pour l'installer il suffit d'exécuter ***pip install mpi4py***. Vous pourrez utiliser l'IDE de votre choix pour écrire les programmes ci-dessous.

### 1.2 L'initialisation et l'exécution

Un communicateur est défini par un groupe de processus et un environnement de communication géré par MPI. Le premier communicateur **COMM_WORLD** correspond à l'ensemble des processus définis au lancement du programme. Les suivants seront construits à partir d'un communicateur parent.


Un programme MPI débute par une initialisation du contexte MPI et permet de définir ce premier communicateur et également l'identifiant **rank** de chaque processus. 

C'est grâce à cet identifiant qu'on pourra écrire qu'un seul programme mais en différentiant les instructions de chaque processus par une condition sur son **rank**.


In [4]:
conda install mpi4py

error: incomplete escape \U at position 28

In [3]:
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

RuntimeError: cannot load MPI library
Could not find module 'C:\Users\ahmed\AppData\Roaming\Python\DLLs' (or one of its dependencies). Try using the full path with constructor syntax.
Could not find module 'C:\Users\ahmed\AppData\Roaming\Python\Library\bin' (or one of its dependencies). Try using the full path with constructor syntax.
Could not find module 'c:\Users\ahmed\anacondas3\DLLs\impi.dll' (or one of its dependencies). Try using the full path with constructor syntax.
Could not find module 'c:\Users\ahmed\anacondas3\DLLs\msmpi.dll' (or one of its dependencies). Try using the full path with constructor syntax.
Could not find module 'c:\Users\ahmed\anacondas3\Library\bin\impi.dll' (or one of its dependencies). Try using the full path with constructor syntax.
Could not find module 'c:\Users\ahmed\anacondas3\Library\bin\msmpi.dll' (or one of its dependencies). Try using the full path with constructor syntax.
Could not find module 'impi.dll' (or one of its dependencies). Try using th

Le premier programme *Hello World* 

In [5]:
%%writefile hello.py

from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
nbprocs = comm.Get_size()

print("Hello World de ",rank," sur",nbprocs)


Overwriting hello.py


L'exécution du programme est géré par mpi via la commande *mpirun*. Elle permet de définir le nombre de processus qui vont être créés sur quels processeurs et sur combien de coeurs.

In [4]:
!mpirun -np 4 python3 ./hello.py

Hello World de  0  sur 4
Hello World de  2  sur 4
Hello World de  1  sur 4
Hello World de  3  sur 4


Il est également possible lorsqu'on utilise une machine parallèle à mémoire distribuée comme une grappe de PCs de définir un fichier listant les processeurs et le nombre de coeurs de l'exécution paralléle.

In [6]:
%%writefile MesHosts.txt
localhost slots=4

Writing MesHosts.txt


In [7]:
!mpirun -np 4 --hostfile MesHosts.txt python3 ./hello.py

Hello World de  3  sur 4
Hello World de  2  sur 4
Hello World de  0  sur 4
Hello World de  1  sur 4


**Exercice 1 :** 

Ecrivez un programme qui permet à chaque processus d'afficher *Hello World* en indiquant son identifiant et également le nombre total de processus de l'exécution paralléle sachant que la fonction *Get_size()* renvoie le nombre de processus d'un communicateur.

Testez votre programme avec la commande suivante en faisant varier *x* de 1 à 6.

In [8]:
%%writefile hello2.py
from mpi4py import MPI

# Ecrire votre code ici
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
nbprocs = comm.Get_size()

print("Hello World de ",rank," sur",nbprocs)


Writing hello2.py


In [14]:
!mpirun -np 6 python3 ./hello2.py

Hello World de  1  sur 6
Hello World de  2  sur 6
Hello World de  3  sur 6
Hello World de  5  sur 6
Hello World de  4  sur 6
Hello World de  0  sur 6


### 1.3 Les communications point-à-point en mode bloquant

La première communication en MPI est une communication bloquante entre un émetteur et un récepteur. Cette communication rend la main lorsque la réception des données est terminée. 

Avec la bibliothèque *mpi4py* une distinction est faite en fonction du type des données.

1. ssend/recv pour "any python object"
2. Ssend/Recv pour "Memory Buffer"

Les exemples suivants réalisent une communication simple entre deux processus. Il utilise ssend/recv car les données communiquées sont soit un entier soit une liste Python.

In [15]:
%%writefile comm1.py
from mpi4py import MPI
import random

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size  = comm.Get_size()

data = random.randint(1,50)

print("je suis ",rank," data avant =",data)

if rank == 0:
    comm.ssend(data, dest=1, tag=11)
elif rank == 1:
    data = comm.recv(source=0, tag=11)

print("je suis ",rank," data après =",data)


Writing comm1.py


In [16]:
!mpirun -np 4 python3 ./comm1.py

je suis  0  data avant = 39
je suis  1  data avant = 28
je suis  3  data avant = 8
je suis  3  data après = 8
je suis  2  data avant = 42
je suis  2  data après = 42
je suis  0  data après = 39
je suis  1  data après = 39


In [17]:
%%writefile comm2.py
from mpi4py import MPI
import random

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size  = comm.Get_size()

tab = [random.randint(1,50) for _ in range(10)]

print("je suis ",rank," tab avant=",tab)

if rank==2:
    comm.ssend(tab,3,10)
elif rank==3:
    tab = comm.recv(source=2,tag=10)

print("je suis ", rank," tab après=",tab)

Writing comm2.py


In [19]:
!mpirun -np 4 python3 ./comm2.py

je suis  0  tab avant= [35, 12, 23, 49, 1, 35, 15, 44, 9, 21]
je suis  0  tab après= [35, 12, 23, 49, 1, 35, 15, 44, 9, 21]
je suis  2  tab avant= [38, 19, 38, 44, 11, 12, 32, 17, 24, 43]
je suis  3  tab avant= [47, 12, 13, 31, 21, 35, 28, 27, 50, 2]
je suis  2  tab après= [38, 19, 38, 44, 11, 12, 32, 17, 24, 43]
je suis  3  tab après= [38, 19, 38, 44, 11, 12, 32, 17, 24, 43]
je suis  1  tab avant= [27, 5, 38, 16, 46, 23, 47, 42, 7, 42]
je suis  1  tab après= [27, 5, 38, 16, 46, 23, 47, 42, 7, 42]


**Exercice 2 :**
Complétez le code suivant pour que le processus d'identifiant 0 transmette à tous les autres processus le contenu de *data*. Attention la transmission inclut l'envoi et la réception.

In [4]:
%%writefile comm3.py
from mpi4py import MPI
import random
import sys

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size  = comm.Get_size()

data = random.randint(1,50)

print("je suis ",rank," data avant =",data)


if rank==0:
    # à compléter
    for i in range(1,size):
        comm.ssend(data,i,10)
else:
    # à compléter
    data = comm.recv(source=0,tag=10)
    
print("je suis ",rank," data après =",data)

Overwriting comm3.py


Le processus d'identifiant 0 joue un rôle particulier. La règle est de ne pas spécifier en dur dans son code le processus qui sera à l'origine des communications. Ce processus est appelé *root* et est donné en argument en ligne de commande. Il faut donc écrire son code de manière à ce que ça fonctionne quelque soit l'identifiant de ce processus root. 
Modifiez votre code afin que ce soit le processus *root* qui transmette sa valeur à tous les autres.

Pour récupérer la valeur de *root* en ligne de commande vous pouvez modifier le début de votre code de la manière suivante :

In [5]:
from mpi4py import MPI
import random
import sys

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size  = comm.Get_size()

root=int(sys.argv[1])
data = random.randint(1,50)

print("je suis ",rank," data avant =",data)


if rank==root:
    # à compléter
    for i in range(1,size):
        comm.ssend(data,i,10)
else:
    # à compléter
    data = comm.recv(source=0,tag=10)
    
print("je suis ",rank," data après =",data)

RuntimeError: cannot load MPI library
Could not find module 'C:\Users\ahmed\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\DLLs' (or one of its dependencies). Try using the full path with constructor syntax.
Could not find module 'C:\Users\ahmed\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Library\bin' (or one of its dependencies). Try using the full path with constructor syntax.
Could not find module 'C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\DLLs\impi.dll' (or one of its dependencies). Try using the full path with constructor syntax.
Could not find module 'C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\DLLs\msmpi.dll' (or one of its dependencies). Try using the full path with constructor syntax.
Could not find module 'C:\Program Files\WindowsApps\PythonSoftwareFoundation.Pyt

Et votre lancement consistera à ajouter l'argument demandé :

In [5]:
!mpirun -np 4 python3 comm3.py 1

je suis  0  data avant = 35
je suis  1  data avant = 21
je suis  1  data après = 35
je suis  2  data avant = 34
je suis  3  data avant = 13
je suis  2  data après = 35
je suis  0  data après = 35
je suis  3  data après = 35


Il est possible également d'échanger des données de type *memory buffer* en utilisant la librairie *numpy*. Dans ce cas les fonctions MPI utilisées sont Ssend/Recv.
Les exemples suivants sont très similaires aux précédents mais utilisent *numpy* pour générer et manipuler les données.

In [34]:
%%writefile comm4.py
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

data = np.random.randint(0,50,1)

print("je suis",rank, " data avant =",data[0])

if rank == 0:
    comm.Ssend(data, dest=1, tag=11)
elif rank == 1:
    comm.Recv(data, source=0, tag=11)

print("je suis",rank, " data après =",data[0])


Overwriting comm4.py


In [35]:
!mpirun -np 4 python3 comm4.py 1 2

je suis 3  data avant = 26
je suis 3  data après = 26
je suis 2  data avant = 35
je suis 2  data après = 35
je suis 0  data avant = 24
je suis 1  data avant = 30
je suis 1  data après = 24
je suis 0  data après = 24


In [36]:
%%writefile comm5.py
from mpi4py import MPI
import numpy as np
import sys

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

src = int(sys.argv[1])
dst = int(sys.argv[2])

buffer = np.random.randint(0,10,size=10)

print("je suis",rank, " buffer avant =",buffer)

if rank == src:
    comm.Ssend(buffer, dest=dst, tag=11)
elif rank == dst:
    comm.Recv(buffer, source=src, tag=11)

print("je suis",rank, " buffer après =",buffer)

Overwriting comm5.py


In [37]:
!mpirun -np 4 python3 comm5.py 1 2

je suis 2  buffer avant = [7 8 2 9 2 0 4 2 8 4]
je suis 0  buffer avant = [4 3 2 1 4 8 3 6 3 9]
je suis 0  buffer après = [4 3 2 1 4 8 3 6 3 9]
je suis 1  buffer avant = [5 9 0 6 8 4 7 3 2 9]
je suis 1  buffer après = [5 9 0 6 8 4 7 3 2 9]
je suis 2  buffer après = [5 9 0 6 8 4 7 3 2 9]
je suis 3  buffer avant = [3 5 3 0 0 6 6 6 7 0]
je suis 3  buffer après = [3 5 3 0 0 6 6 6 7 0]


**Exercice 3 :** Ecrivez le programme pour que chaque processus génére un tableau de taille $n$ (donné en argument en ligne de commande) et ensuite réalise un échange circulaire des tableaux ($P_0$ envoie à $P_1$ qui envoie à $P_2$ etc et $P_0$ recoit du dernier qui recoit de l'avant dernier etc). 

In [2]:
%%writefile echange_circ.py
from mpi4py import MPI
import numpy as np
import sys

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

n = int(sys.argv[1])

monTab = np.random.randint(0,50,n)
print("je suis ", rank, " monTab=", monTab)


tabRecu = np.zeros(n,dtype='int') # Pour recevoir le tableau du voisin de gauche

# A compléter.

dest = (rank+1)%size
src = (rank-1)%size

if rank%2==0:
    comm.Ssend(monTab, dest=dest, tag=11)
    comm.Recv(tabRecu, source=src, tag=11)
else:
    comm.Recv(tabRecu, source=src, tag=11)
    comm.Ssend(monTab, dest=dest, tag=11)

print("je suis ", rank, " tabRecu=", tabRecu)



Overwriting echange_circ.py


In [3]:
!mpirun -np 4 python3 echange_circ.py 10

je suis  1  monTab= [44 19 35 20  8 23  2 28  9  6]
je suis  0  monTab= [28  7 11 21 12 10  2 26 19 34]
je suis  3  monTab= [13 30 37 30 24  5 37 20 43 34]
je suis  2  monTab= [19 39 40  1 24 23  7 11 24 23]
je suis  1  tabRecu= [28  7 11 21 12 10  2 26 19 34]
je suis  0  tabRecu= [13 30 37 30 24  5 37 20 43 34]
je suis  2  tabRecu= [44 19 35 20  8 23  2 28  9  6]
je suis  3  tabRecu= [19 39 40  1 24 23  7 11 24 23]


### 1.4 Les communications point-à-point en mode non bloquant

Les communications en mode non bloquant rendent la main et permettent la poursuite des instructions avant que la communication soit terminée. 

Par exemple pour un émetteur, si la donnée est transmise par une communication non bloquante, que se passe-t-il si la donnée est modifiée par cet émetteur avant que la communication ne soit terminée ?

Pour être sûr du comportement du programme des fonctions supplémentaires sont disponibles pour tester si la communication est terminée. Ainsi on peut s'assurer de ne rien modifier avant que la transmission soit complète.

Il y a toujours une distinction en fonction du type des données.

1. issend/irecv pour "any python object"
2. Issend/Irecv pour "Memory Buffer"

Les fonctions test et wait permettent de vérifier la fin de la communication. Les deux exemples ci-dessous illustrent le fonctionnement et la différence entre ces deux fonctions.

In [7]:
%%writefile comm6.py
from mpi4py import MPI
import random

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

data = random.randint(1,50)

print("je suis ",rank," data=",data)

if rank == 0:
    req = comm.issend(data, dest=1, tag=11)
elif rank == 1:
    req = comm.irecv(source=0, tag=11)
    rcv = 0
    while rcv == 0:
        [rcv, data] = req.test()

print("je suis", rank, " data=", data)

Writing comm6.py


In [ ]:
%%writefile comm7.py
from mpi4py import MPI
import random

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

data = random.randint(1,50)

print("je suis ",rank," data=",data)

if rank == 0:
    req = comm.issend(data, dest=1, tag=11)
elif rank == 1:
    req = comm.irecv(source=0, tag=11)
    data = req.wait()

print("je suis",rank, " data=",data)

**Exercice 4 :** Expliquez la différence entre *test* et *wait*. Notez que les fonctions *issend* et *irecv* renvoient une information (*req* dans l'exemple) qui correspond à la communication en cours.

La différence entre les fonctions test() et wait() est que test() permet de verifier si la communication entre deux processus a eu lieu alors que wait() permet de bloquer le programme jusqu'a ce que la communication soit terminée.

**Exercice 5 :** Ecrivez le code pour l'échange circulaire en utilisant des *memory buffers* et des communications non bloquantes.

In [1]:
%%writefile echange_circ2.py
from mpi4py import MPI
import numpy as np
import sys

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

n = int(sys.argv[1])
monTab = np.random.randint(1,50,n)

print("je suis ",rank," monTab=",monTab)


tabRecu = np.zeros(n,dtype='int')

# A compléter.
dest = (rank+1)%size
src = (rank-1)%size


comm.Issend(monTab, dest=dest, tag=11)
req = comm.Irecv(tabRecu, source=src, tag=11)
req.wait()


print("je suis",rank, " tabRecu=",tabRecu)


Writing echange_circ2.py


In [5]:
!mpirun -np 4 python3 echange_circ2.py 10

je suis  1  monTab= [43 20 26 13  1 47 14  9 14 38]
je suis  0  monTab= [ 2 22 49 43 15 31 34 44 46 28]
je suis  3  monTab= [ 5 46 41  3 28 40  1 46 17 45]
je suis 0  tabRecu= [ 5 46 41  3 28 40  1 46 17 45]
je suis 1  tabRecu= [ 2 22 49 43 15 31 34 44 46 28]
je suis  2  monTab= [19 21 48 34 34 33 41 17 15 26]
je suis 2  tabRecu= [43 20 26 13  1 47 14  9 14 38]
je suis 3  tabRecu= [19 21 48 34 34 33 41 17 15 26]


**Exercice 6:** Ecrivez le code qui permet à un processus *root* de construire un tableau et de le distribuer par morceaux à chaque processus y compris lui même. On supposera que la taille du tableau est divisible par le nombre de processus.

Par exemple si $P_0$ génère le tableau de taille 20 [0 7 3 1 4 4 1 0 9 6 3 5 9 4 9 6 4 4 0 9]$ à la fin de l'exécution sur 4 processus on aura 

sur $P_0$ : 0 7 3 1 4 

sur $P_1$ : 4 1 0 9 6 

sur $P_2$ : 3 5 9 4 9 

sur $P_3$ : 6 4 4 0 9

Autre exemple si $P_2$ génère le tableau de taille 24 [6 2 7 5 9 2 8 2 6 3 0 1 6 2 5 4 6 8 9 2 1 7 2 4] à la fin de l'exécution sur 4 processus on aura

sur $P_0$ : 6 2 7 5 9 2 

sur $P_1$ : 8 2 6 3 0 1 

sur $P_2$ : 6 2 5 4 6 8 

sur $P_3$ : 9 2 1 7 2 4


In [ ]:
%%writefile distribution.py
from mpi4py import MPI
import numpy as np
import sys

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

root = int(sys.argv[1])
n = int(sys.argv[2])

if (n%size !=0 ):
    if (rank==root):
    print("Il faut que la taille du tableau soit divisible par le nombre de processus")
    exit()

nlocal = n // size

monTab = np.zeros(nlocal,dtype='int')

if rank==root:
    tableau = np.random.randint(0,10,size=n)
    print("je suis",rank, " le tableau à découper =",tableau)
    
    # A compléter à partir d'ici
    for p in range(size):
        start = p*nlocal
        end = (p+1)*nlocal
        morceau = tableau[start:end]

        if p = root:
            montab[:]=morceau

        else:
            comm.Ssend(morceau,p,tag = 11)

else:
    comm.Rcev(monTab,source=root,tag=11)

print("Processus", rank, "a reçu :", monTab)




Writing distribution.py


In [15]:
pip install mpi4py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 18.9 MB/s eta 0:00:0000:010:01


In [16]:
%%writefile distribution2.py
from mpi4py import MPI
import numpy as np
import sys

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

root = int(sys.argv[1])
n = int(sys.argv[2])

if (n%size !=0 ):
    if (rank==root):
        print("Il faut que la taille du tableau soit divisible par le nombre de processus")
        exit()
    

nlocal = n // size

monTab = np.zeros(nlocal,dtype='int')

if rank==root:
    tableau = np.random.randint(0,10,size=n)
    print("je suis",rank, " le tableau à découper =",tableau)

else:
    tableau = None

comm.Scatter(tableau,monTab,root= root)

print("Processus", rank," a reçu :", monTab)




Overwriting distribution2.py


In [23]:
import os
print(os.cpu_count())


2


In [22]:
%%bash
export OMPI_ALLOW_RUN_AS_ROOT=1
export OMPI_ALLOW_RUN_AS_ROOT_CONFIRM=1
mpirun --oversubscribe -np 4 python3 distribution2.py 0 16


je suis 0  le tableau à découper = [5 1 6 0 7 3 5 5 2 0 8 7 9 4 9 1]
Processus 1  a reçu : [7 3 5 5]
Processus 0  a reçu : [5 1 6 0]
Processus 2  a reçu : [2 0 8 7]
Processus 3  a reçu : [9 4 9 1]


**Exercice 7 :** Comment adapter ce code si la taille du tableau n'est pas divisible par le nombre de processus ?

In [ ]:
%%writefile distribution_nondivi.py
from mpi4py import MPI
import numpy as np
import sys

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

root = int(sys.argv[1])
n = int(sys.argv[2])

if (n%size !=0 ):
    if (rank==root):
    print("Il faut que la taille du tableau soit divisible par le nombre de processus")
    exit()

nlocal = n // size

monTab = np.zeros(nlocal,dtype='int')

if rank==root:
    tableau = np.random.randint(0,10,size=n)
    print("je suis",rank, " le tableau à découper =",tableau)
    
    # A compléter à partir d'ici
    for p in range(size):
        start = p*nlocal
        end = (p+1)*nlocal
        morceau = tableau[start:end]

        if p = root:
            montab[:]=morceau

        else:
            comm.Ssend(morceau,p,tag = 11)

else:
    comm.Rcev(monTab,source=root,tag=11)

print("Processus", rank, "a reçu :", monTab)




In [ ]:
!mpirun -np 4 python3 distribution_nondivi.py 0 10